<a href="https://colab.research.google.com/github/VikramAchuthan/hello-world/blob/master/FinishedHockeyDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [2]:
#Retrieve data of all D1 and D2 NCAA women's hockey colleges
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Th01ggWcHbjQGE6HBYstuiGHbO4JTcq8FnbKgL5PzXc/edit#gid=0')

collegesSheet = wb.worksheet('NCAA D1 and D2 Schools')

collegesData = collegesSheet.get_all_values()
print(collegesData)

[['Gender', 'Level', 'College', 'State', 'League'], ['Women', 'NCAA Division I', 'Bemidji State University', 'Minnesota', 'Western Collegiate Hockey Association (WCHA)'], ['Women', 'NCAA Division I', 'Boston College', 'Massachusetts', 'Hockey East (HEA)'], ['Women', 'NCAA Division I', 'Boston University', 'Massachusetts', 'Hockey East (HEA)'], ['Women', 'NCAA Division I', 'Brown University', 'Rhode Island', 'Eastern College Athletic Conference (ECAC)'], ['Women', 'NCAA Division I', 'Clarkson University', 'New York', 'Eastern College Athletic Conference (ECAC)'], ['Women', 'NCAA Division I', 'Colgate University', 'New York', 'Eastern College Athletic Conference (ECAC)'], ['Women', 'NCAA Division I', 'Connecticut College', 'Connecticut', 'Hockey East (HEA)'], ['Women', 'NCAA Division I', 'University of Connecticut', 'Connecticut', 'Hockey East (HEA)'], ['Women', 'NCAA Division I', 'Cornell University', 'New York', 'Eastern College Athletic Conference (ECAC)'], ['Women', 'NCAA Division I'

In [3]:
# Import initial libraries
import pandas as pd
import numpy as np

print("Completed")

Completed


In [4]:
#Store colleges in panda dataframe

collegeDF = pd.DataFrame(collegesData)
collegeDF.columns = collegeDF.iloc[0]
collegeDF = collegeDF.iloc[1:]

collegeDF.head(40)
print(collegeDF.columns)

Index(['Gender', 'Level', 'College', 'State', 'League'], dtype='object', name=0)


In [ ]:

# import html5lib
# from bs4 import BeautifulSoup
# from urllib.request import Request, urlopen

# site= "https://collegecommitments.com/CommitList.aspx?x7cq9=FEMALE"
# hdr = {'User-Agent': 'Mozilla/5.0'}
# req = Request(site,headers=hdr)
# page = urlopen(req)
# soup = BeautifulSoup(page)
# print(soup.get_text())




In [5]:
# #Retrieve data of all players who committed to schools to play WIH


import requests
import lxml.html as lh
#website with data
url='https://collegecommitments.com/CommitList.aspx?x7cq9=FEMALE'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')


tr_elements_new = tr_elements[5:]



In [ ]:
col=[]
i=0
tableHead = ['Date','Player Name', 'Pos.','College','Current Team', 'Starting Year']

#For each row, store each first element (header) and an empty list
for x in range(0,6):
    i+=1
    name=tableHead[x]
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

for j in range(1,len(tr_elements_new)):
    #T is our j'th row
    T=tr_elements_new[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=6:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

#create panda dataframe of all players who committed 
playerDataDict={title:column for (title,column) in col}
playerDataDF=pd.DataFrame(playerDataDict)
playerDataDF.head(950)


1:"Date"
2:"Player Name"
3:"Pos."
4:"College"
5:"Current Team"
6:"Starting Year"


,Date,Player Name,Pos.,College,Current Team,Starting Year
0,06/28/20,"SWEETAPPLE, Mary",D,Mercyhurst University,Ridley College (JWHL),2021-22
1,06/26/20,"MURDOCH, Raine",LD,Union College,Edmonton Pandas MAAA (AMAAA),2022-23
2,06/25/20,"TATTERSALL, Joanne",D,Brown University,New Hampton HS (NEDPS),2022-23
3,06/20/20,"PEDONE, Molly",F,Robert Morris University,Shattuck St. Mary's Prep (UpMW),2020-21
4,06/18/20,"DWYER, Grace",D,Cornell University,Philadelphia Jr. Flyers U16 (ONTC),2022-23
...,...,...,...,...,...,...
669,12/08/16,"BURTON, Adelaide",F,University of Minnesota,MN Jr Whitecaps Black U19 (USA),2020-21
670,12/05/16,"MATTISON, Jorden",G,University of Maine,Etobicoke Jr. Dolphins (PWHL),2020-21
671,11/11/16,"O'BRIEN, Casey",F,University of Wisconsin,Shattuck St. Mary's Prep (UpMW),2020-21
672,10/18/16,"MURPHY, Abbey",F,University of Minnesota,Chicago Mission U19 (HPHL),2020-21


In [6]:
playerSheet = wb.worksheet('Players Information')

playerData = playerSheet.get_all_values()

playerDataDF = pd.DataFrame(playerData)
playerDataDF.columns = playerDataDF.iloc[0]
playerDataDF = playerDataDF.iloc[1:]

playerDataDF.head(950)



,Date,Player Name,Position,College,Current Team,Starting Year
1,05/06/20,"PRESSEAULT, Michaela",F,Nipissing University,Barrie Sharks (PWHL),2020-21
2,05/06/20,"LEBEL, Madyson",F,Nipissing University,,2020-21
3,05/06/20,"NEFF, Emma",LD,Nipissing University,Calgary Fire MAAA (AMAAA),2020-21
4,05/06/20,"LABERGE, Elizabeth",D,Nipissing University,Sudbury Lady Wolves MAA (LLMAA),2022-23
5,05/06/20,"KOSEC, Tessa",D,Nipissing University,OHA Tardiff (Maroon (NCWHL),2020-21
...,...,...,...,...,...,...
912,11/13/14,"MAROLT, Sylvia",F,Bemidji State University,,2015-16
913,11/13/14,"RASMUSSEN, Dana",F,Ohio State University,St. Cloud State University (WCHA),2015-16
914,11/13/14,"O'NEILL, Daria",D,University of Vermont,Mount Royal University (CW),2015-16
915,11/13/14,"PUTIGNA, Christina",F,Providence College,Boston Pride (NWHL),2015-16


In [7]:
#Filter to include only Players who committed to D1 and D2 schools
#create data frame of those players
#print(playerDataDF.iloc[0])
print(len(playerDataDF))
filteredPlayers = []
for x in range(len(playerDataDF)):
  #print(playerDataDF.iloc[x][1])# == 'MURRAY,Lydia'):
  #  print("YES!")
  committedCollege =  playerDataDF.iloc[x][3]
  nameList = playerDataDF.iloc[x][1].split(",")
  playerDataDF.iloc[x][1] = nameList[1] + " " + nameList[0].title()
 # print("COLLEGE:",committedCollege)
  if collegeDF['College'].str.contains(committedCollege.strip()).any():
    if(playerDataDF.iloc[x][1] == 'Lydia Murray'): 
      print("YES!!!!!")
    filteredPlayers.append([playerDataDF.iloc[x][0],playerDataDF.iloc[x][1],playerDataDF.iloc[x][2],playerDataDF.iloc[x][3],playerDataDF.iloc[x][4],playerDataDF.iloc[x][5]])
print(filteredPlayers)
filteredPlayersDF = pd.DataFrame(filteredPlayers)
filteredPlayersDF.columns = playerDataDF.columns
#filteredPlayersDF = filteredPlayersDF.iloc[1:]
filteredPlayersDF.head(600)


916
[['04/29/20', ' Elizabeth Parento', 'D', 'University of Vermont', '', '2020-21'], ['03/27/20', ' Emilie Prive', 'F', 'Franklin Pierce University', 'NAHA White (JWHL)', '2020-21'], ['03/10/20', ' Ashlyn Lewis', 'G', 'Rensselaer Polytechnic Institute', 'NAHA White (JWHL)', '2021-22'], ['03/04/20', ' Rose Jeneault', 'RD', 'Brown University', 'Nepean Jr. Wildcats (PWHL)', '2020-21'], ['03/04/20', ' Riley Smith', 'LD', 'Robert Morris University', 'St. Albert Slash MAAA (AMAAA)', '2021-22'], ['02/24/20', ' Holly Kubichek', 'F', 'Post University', 'Detroit Little Ceasars U19 (HPHL)', '2020-21'], ['02/24/20', ' Elle Mckenna', 'D', 'Post University', 'Detroit Little Ceasars U19 (HPHL)', '2020-21'], ['02/24/20', ' Emma Decorby', 'LD', 'Yale University', 'Regina Rebels MAAA (SFMAAA)', '2021-22'], ['02/20/20', ' Grace Glasrud', 'F', 'Post University', '', '2020-21'], ['02/20/20', ' Chloe Corbin', 'F', 'Lindenwood University', '', '2020-21'], ['02/20/20', ' Olivia Rinzel', 'G', 'Union College',

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Date,Player Name,Position,College,Current Team,Starting Year
0,04/29/20,Elizabeth Parento,D,University of Vermont,,2020-21
1,03/27/20,Emilie Prive,F,Franklin Pierce University,NAHA White (JWHL),2020-21
2,03/10/20,Ashlyn Lewis,G,Rensselaer Polytechnic Institute,NAHA White (JWHL),2021-22
3,03/04/20,Rose Jeneault,RD,Brown University,Nepean Jr. Wildcats (PWHL),2020-21
4,03/04/20,Riley Smith,LD,Robert Morris University,St. Albert Slash MAAA (AMAAA),2021-22
...,...,...,...,...,...,...
557,11/13/14,Sylvia Marolt,F,Bemidji State University,,2015-16
558,11/13/14,Dana Rasmussen,F,Ohio State University,St. Cloud State University (WCHA),2015-16
559,11/13/14,Daria O'Neill,D,University of Vermont,Mount Royal University (CW),2015-16
560,11/13/14,Christina Putigna,F,Providence College,Boston Pride (NWHL),2015-16


In [8]:
#add to google spreadsheet
# worksheet = wb.add_worksheet(title="Filtered Hockey Players", rows="1000", cols="20")
# worksheet.update([filteredPlayersDF.columns.values.tolist()] + filteredPlayersDF.values.tolist())

#create a dictionary matching college to hockey data for that college
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Th01ggWcHbjQGE6HBYstuiGHbO4JTcq8FnbKgL5PzXc/edit#gid=0')

collegesSheet = wb.worksheet('College Statistics Websites')

collegeStatWebsiteList = collegesSheet.get_all_values()
collegeStatWebsiteDict = {}
for college in collegeStatWebsiteList:
  collegeStatWebsiteDict[college[0]] = college[1]
print(collegeStatWebsiteDict)




{'Lindenwood University': 'http://www.collegehockeystats.net/1819/teamstats/linw', 'Mercyhurst University': 'http://www.collegehockeystats.net/1819/teamstats/mrcw', 'Pennsylvania State University': 'http://www.collegehockeystats.net/1819/teamstats/psuw', 'Rochester Institute of Technology': 'http://www.collegehockeystats.net/1819/teamstats/ritw', 'Robert Morris University': 'http://www.collegehockeystats.net/1819/teamstats/rmuw', 'Syracuse University': 'http://www.collegehockeystats.net/1819/teamstats/syrw', 'Brown University': 'http://www.collegehockeystats.net/1819/teamstats/brnw', 'Cornell University': 'http://www.collegehockeystats.net/1819/teamstats/corw', 'Darthmouth College': 'http://www.collegehockeystats.net/1819/teamstats/darw', 'Harvard University': 'http://www.collegehockeystats.net/1819/teamstats/harw', 'Princeton University': 'http://www.collegehockeystats.net/1819/teamstats/prnw', 'Yale University': 'http://www.collegehockeystats.net/1819/teamstats/yalw', 'Clarkson Unive

In [10]:
#ask user to input season for which stats are required
#For each player in filteredPlayersDF, see if they played in the inputted season
#Get their statistics, and add it to a dataframe for that season. 
from datetime import datetime

yearToCodeDict = {'2014':'1415','2015':'1516','2016':'1617','2017':'1718','2018':'1819','2019':'1920','2020':'2021'}

experience = int(input("Enter the experience: 0,1,2,3: "))
#websiteYear = mostRecentSeason[2:4] + mostRecentSeason[5:]
#inputYear = mostRecentSeason
#mostRecentSeason = int(mostRecentSeason[:4])

count=0
playerProfileDict = {}
for season in range(2014,2019):
  for x in range(len(filteredPlayersDF)):
    startingYear = int(filteredPlayersDF.iloc[x][5][:4]) #entry year
    if (startingYear == season-experience) and filteredPlayersDF.iloc[x][3] in collegeStatWebsiteDict.keys() and filteredPlayersDF.iloc[x][2] != 'G':
      playerName = filteredPlayersDF.iloc[x][1] 
      #print("Player Name: {}    Starting Year: {}      Season: {}".format(playerName,startingYear,season))
      college = filteredPlayersDF.iloc[x][3]
      collegeWebsite = collegeStatWebsiteDict[college]
      collegeWebsite = collegeWebsite.replace('1819', yearToCodeDict[str(season)])
    #  print("Website",collegeWebsite)
    # print("website: {}".format(collegeWebsite))
      statisticForSeasonDF = pd.read_html(collegeWebsite)
    # print("COLLEGE: {}      {}".format(college,playerName))
      statisticForSeasonDF = pd.DataFrame(statisticForSeasonDF[1])
      selectedPlayerStatistics = statisticForSeasonDF.loc[statisticForSeasonDF[1] == playerName.strip()].to_string()
  
      indexofname = selectedPlayerStatistics.find(playerName)
      selectedPlayerStatistics = selectedPlayerStatistics[indexofname:]
      #itemize full list into individual stats
      selectedPlayerStatistics = selectedPlayerStatistics.split()
      selectedPlayerStatistics = selectedPlayerStatistics[2:]
      #convert dates (string object) to datetime objects
      committed_date_time_str = filteredPlayersDF.iloc[x][0]
      #print(committed_date_time_str)
      committed_date_time_obj = datetime.strptime(committed_date_time_str, '%m/%d/%y') 
      entry_date_time_str = '09/01/'+filteredPlayersDF.iloc[x][5][2:4]
      entry_date_time_obj = datetime.strptime(entry_date_time_str, '%m/%d/%y') 
      difference = entry_date_time_obj-committed_date_time_obj
      elapsedTimeMonths = (difference.total_seconds())/2592000
      playerProfileList = [playerName,college,entry_date_time_obj,committed_date_time_obj,round(elapsedTimeMonths,2),season]

      x = 0
      integerIndexes = [2,3,4,5,7,8,9,11,12,13,14,15,17,18,19,21,22,23,24,25]
      for statistic in selectedPlayerStatistics:
        if x in integerIndexes and statistic !='NaN': 
          statistic = int(statistic)
        x +=1
        playerProfileList.append(statistic)
      #print(len(playerProfileList))
      if len(playerProfileList) < 31:
      # print("YEYEYEYEY",playerName,len(playerProfileList))
        count+=1
        continue
    # playerProfileList.append(filteredPlayersDF.iloc[x][5]) #appending year committed in string format  for convenience
      playerProfileDict[playerName] = playerProfileList
print("HI",playerProfileDict)

print("count:",count, "       ",len(playerProfileDict))
    
    
    # myList = statisticForSeasonDF[1].to_list()
    # print(myList)

    # for each in myList:
    #   print("Player:",each,len(each), "    ",playerName, len(playerName))
    #   if each == playerName.strip():
    #     print("YES")
    #print(statisticForSeasonDF)
    # teamRosterList = statisticForSeasonDF[1].iloc[:, 1].to_list()
    # playerStatistic = statisticForSeasonDF[1][1]
    # statisticForSeasonDF[1].drop(statisticForSeasonDF[1].index[0])
    # print(statisticForSeasonDF[1].head(20))

  # print(playerStatistic)
      
    #print("*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*$*")    
  
# raw_data = {'name': ['Willard Morris', 'Al Jennings', 'Omar Mullins', 'Spencer McDaniel'],
# 'age': [20, 19, 22, 21],
# 'favorite_color': ['blue', 'blue', 'yellow', "green"],
# 'grade': [88, 92, 95, 70]}

# df = pd.DataFrame(raw_data)
# df.head()
# df.loc[df['favorite_color'] == 'yellow']
# print(df)
34



print(playerProfileDict)
playerProfileDF = pd.DataFrame.from_dict(playerProfileDict, orient = 'index')
playerProfileDF
playerProfileDF.columns = ['Player Name','College','Entry Year','Date Committed', 'Elapsed Months','Statistics From','Position',	'YR	','Overall GP',	'Overall G'	,'Overall A',	'Overall PTS'	,'Overall PEN/MIN',	'Overall PP',	'Overall SH'	,
                          'Overall GW'	,'Overall +/-',	'Overall SOG',	'Conference GP',	'Conference G',	'Conference A',	'Conference PTS',	'Conference PEN/MIN',	'Conference PP',	'Conference SH',	'Conference GW'	,'Conference +/-',	'Conference SOG',	'Career GP',	'Career G',	'Career A',	'Career PT']
#playerProfileDF[playerProfileDF.POS.str.contains(None) == False]
playerProfileDF.head(100)


Enter the experience: 0,1,2,3: 1
HI {' Nicole Schammel': [' Nicole Schammel', 'University of Minnesota', datetime.datetime(2015, 9, 1, 0, 0), datetime.datetime(2015, 5, 15, 0, 0), 3.63, 2016, 'F', 'JR', 39, 6, 6, 12, '4/8', 0, 0, 2, '+2', 88, 28, 6, 4, 10, '4/8', 0, 0, 2, '+5', 63, 75, 17, 17, 34], ' Corbin Welsh': [' Corbin Welsh', 'Lindenwood University', datetime.datetime(2015, 9, 1, 0, 0), datetime.datetime(2015, 5, 1, 0, 0), 4.1, 2016, 'F', 'SO', 18, 1, 0, 1, '3/6', 0, 0, 0, '-5', 8, 11, 0, 0, 0, '1/2', 0, 0, 0, '-2', 5, 55, 1, 3, 4], ' Sierra Smith': [' Sierra Smith', 'University of Minnesota', datetime.datetime(2015, 9, 1, 0, 0), datetime.datetime(2015, 3, 4, 0, 0), 6.03, 2016, 'F', 'SO', 5, 0, 0, 0, '1/2', 0, 0, 0, '-1', 1, 2, 0, 0, 0, 'NaN', 0, 0, 0, 'E', 1, 15, 0, 0, 0], ' Sarah Stuehr': [' Sarah Stuehr', 'Syracuse University', datetime.datetime(2015, 9, 1, 0, 0), datetime.datetime(2015, 2, 11, 0, 0), 6.73, 2016, 'F', 'SO', 31, 1, 0, 1, '4/8', 0, 0, 0, '-1', 6, 18, 1, 0, 1, '

,Player Name,College,Entry Year,Date Committed,Elapsed Months,Statistics From,Position,YR\t,Overall GP,Overall G,Overall A,Overall PTS,Overall PEN/MIN,Overall PP,Overall SH,Overall GW,Overall +/-,Overall SOG,Conference GP,Conference G,Conference A,Conference PTS,Conference PEN/MIN,Conference PP,Conference SH,Conference GW,Conference +/-,Conference SOG,Career GP,Career G,Career A,Career PT
Nicole Schammel,Nicole Schammel,University of Minnesota,2015-09-01,2015-05-15,3.63,2016,F,JR,39,6,6,12,4/8,0,0,2,+2,88,28,6,4,10,4/8,0,0,2,+5,63,75,17,17,34
Corbin Welsh,Corbin Welsh,Lindenwood University,2015-09-01,2015-05-01,4.10,2016,F,SO,18,1,0,1,3/6,0,0,0,-5,8,11,0,0,0,1/2,0,0,0,-2,5,55,1,3,4
Sierra Smith,Sierra Smith,University of Minnesota,2015-09-01,2015-03-04,6.03,2016,F,SO,5,0,0,0,1/2,0,0,0,-1,1,2,0,0,0,NaN,0,0,0,E,1,15,0,0,0
Sarah Stuehr,Sarah Stuehr,Syracuse University,2015-09-01,2015-02-11,6.73,2016,F,SO,31,1,0,1,4/8,0,0,0,-1,6,18,1,0,1,3/6,0,0,0,+2,4,67,1,0,1
Shannon Morris-Reade,Shannon Morris-Reade,Lindenwood University,2015-09-01,2014-12-10,8.83,2016,F,SO,33,3,4,7,6/12,2,0,0,-20,70,20,1,0,1,3/6,1,0,0,-15,39,70,11,19,30
Tirra Lemoine,Tirra Lemoine,Lindenwood University,2015-09-01,2014-12-10,8.83,2016,F,SO,18,1,2,3,7/14,0,0,1,-3,10,10,1,0,1,5/10,0,0,1,-1,7,55,3,2,5
Jenna Rheault,Jenna Rheault,University of New Hampshire,2015-09-01,2014-11-19,9.53,2016,D,SO,35,2,3,5,6/12,0,0,0,-3,26,24,2,3,5,5/10,0,0,0,E,20,71,4,4,8
Nora Maclaine,Nora Maclaine,University of Connecticut,2015-09-01,2014-11-19,9.53,2016,F,SO,33,1,5,6,10/20,0,0,0,-13,39,22,0,4,4,9/18,0,0,0,-11,32,70,2,6,8
Justine Reyes,Justine Reyes,St. Lawrence University,2015-09-01,2014-11-19,9.53,2016,F,SO,36,13,10,23,4/8,3,0,5,+11,88,22,8,7,15,2/4,3,0,3,+10,48,74,25,18,43
Leah Olson,Leah Olson,Brown University,2015-09-01,2014-11-19,9.53,2016,D,SO,29,4,7,11,11/33,1,0,0,-7,52,22,2,5,7,11/33,0,0,0,-10,36,58,6,16,22


In [ ]:
from google.colab import drive
drive.mount('/drive')


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:

# playerProfileDF.plot(x ='Elapsed Months', y='G', kind = 'scatter')
# plt.show()

import matplotlib.pyplot as plt
import plotly.express as px
yVariable = str(input("Enter the statistic you wish to measure against elapsed time:"))
#         yVariable: playerProfileDF[yVariable]
#        }
graphedDataList = []

for key in playerProfileDF['Elapsed Months'].keys(): #key is the player name
  graphedDataList.append([key,playerProfileDF['Position'][key],playerProfileDF['Elapsed Months'][key],playerProfileDF[yVariable][key],playerProfileDF['College'][key],playerProfileDF['Statistics From'][key],playerProfileDF['Overall GP'][key]])

sumX = 0
sumY = 0
# for each in graphedDataList:
#   sumX = sumX + float(each[2])
#   sumY = sumY + int(each[3])
graphedDataList.append(['MEAN','MEAN',sumX/len(graphedDataList),sumY/len(graphedDataList),'NA','NA','NA'])
graphedDataDF = pd.DataFrame(graphedDataList)
graphedDataDF.columns = ['Player Name','Position', 'Elapsed Months', yVariable,'College','Statistics From','Overall GP']
graphedDataDF.head(60)

#df = DataFrame(Data,columns=['Elapsed Months',yVariable])
#graphedDataDF = graphedDataDF.sort_values(by=[yVariable])
# graphedDataDF.plot(x ='Elapsed Months', y=yVariable, kind = 'scatter')
# plt.show()


experienceToClassDict = {0:'Freshmen',1:"Sophomore",2:"Junior",3:'Senior'}


fig = px.scatter(graphedDataDF, x="Elapsed Months", y=yVariable,color = 'Position',hover_name="Player Name",hover_data=["College",'Statistics From','Overall GP'])#hover_data=["continent", "pop"])
#fig.update_traces(marker_color='blue',line_color='red')
fig.update_layout(
    title= yVariable + " vs Elapsed Months, "+experienceToClassDict[experience] + " Hockey Players" ,
 
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.show()
fig.write_html("/drive/My Drive/"+experienceToClassDict[experience]+"HockeyDataset.html")


# import plotly.io as pio
# pio.write_html(fig, file='index.html', auto_open=True)


fig1 = px.histogram(graphedDataDF, x="Elapsed Months", y=yVariable, log_x=True)#,hover_name="Player Name",trendline="ols")#hover_data=["continent", "pop"])
fig1.update_layout(
    title= yVariable + " vs Elapsed Months, "+experienceToClassDict[experience] + " Hockey Players" ,
 
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig1.show()

Enter the statistic you wish to measure against elapsed time:Saves


In [ ]:
pwd

'/content'

In [ ]:
!pip install inquirer
import inquirer
questions = [
  inquirer.List('size',
                message="What size do you need?",
                choices=['Jumbo', 'Large', 'Standard', 'Medium', 'Small', 'Micro'],
            ),
]
answers = inquirer.prompt(questions)
print(answers["size"])

     |████████████████████████████████| 81kB 3.5MB/s 

[?] What size do you need?: Jumbo
 > Jumbo
   Large
   Standard
   Medium
   Small
   Micro



error: ignored

In [ ]:
#upload to spread - skip cell if not necessary

import gspread
from oauth2client.client import GoogleCredentials as GC

from gspread_dataframe import set_with_dataframe
gc = gspread.authorize(GC.get_application_default())

title = 'Stats for ' + inputYear 
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet, playerProfileDF) 



ImportError: ignored